## 1. What Is eval()?

Imagine you have a magic calculator. Instead of punching buttons, you write down a mathematical problem on a piece of paper as a string of text, like "10 * (5 + 2)". You then hand this paper to the calculator, and it instantly gives you back the answer: 70.

In essence, Python's eval() function is like that magic calculator. It takes a string containing a Python expression, and it evaluates it, returning the resulting value.

Definition: eval(expr, globals=None, locals=None) takes a string containing a Python expression, parses and compiles it at runtime, executes it, and returns the result.

Key point: Only expressions are allowed (things that produce a value)—not full statements like for loops, if … else, or import.

## 2. Expressions vs. Statements

What if you write a command on the paper, like "create a new variable called z and set it to 10"? The magic calculator would be confused. It's built to calculate values, not to perform actions.

This is the key limitation of eval(): it cannot handle statements. A statement is an instruction that does something, like assigning a variable, creating a loop, or importing a library.

- Expressions produce a value:

    - Literals: 42, "hello", True

    - Operations: 2 + 3 * 4, x**2 - y

    - Function calls returning something: len("abc"), math.sin(0.5)

- Statements perform actions without producing a direct value:

    - Assignments: x = 10

    - Loops: for i in range(3): …

    - Imports: import os

> Doing eval("x = 10") fails, because x = 10 is a statement, not an expression.

## 3. Basic Usage

In [2]:
# Simple math via eval
eval("2**10")

# Using variables defined in your current namespace
x = 7
eval("x + 3")

10

In [8]:
print(eval(input("Enter the Python Expression: ")))

Enter the Python Expression:  7//2-5*10


-47


## 4. Usage scenario

Imagine you're building a system to validate data. The validation rules are stored in a database or a configuration file and might change often. You don't want to rewrite your code every time a rule changes.

Let's say you have a rule that a product_price must be greater than the cost_to_make plus a minimum_margin.

In [ ]:
# Data for a specific product
product_data = {
    "product_price": 150,
    "cost_to_make": 110,
    "minimum_margin": 30
}

# The rule, stored as a string
rule_string = "product_price > cost_to_make + minimum_margin"

# Here, we passed our product_data dictionary as the second argument to eval().
# This gives eval() a controlled "environment" to work in. It can see and use the keys from the dictionary as variables.
# We can use eval() to check if the rule passes
is_valid = eval(rule_string, product_data)

print(f"Does the product meet the rule? {is_valid}")

Does the product meet the rule? True


## 5. Controlling the Environment
By default, eval() runs in your current global and local namespace—and so it can see all your variables and functions. You can lock it down:

In [5]:
# Only allow names from this dict for globals (no built-ins either)
safe_globals = {"__builtins__": None, "pi": 3.14159}
eval("pi * 2", safe_globals)

6.28318

In [6]:
# Provide separate locals as well
locs = {"radius": 5}
eval("pi * radius**2", safe_globals, locs)

78.53975

#### Think of globals/locals as the pantry shelves you let the chef (eval) access. Give only the ingredients (variables/functions) you trust.

## 6. The DANGER Zone

- This is the most important part of the lesson. Using eval() can be incredibly dangerous, especially if you pass it a string that comes from a user. Think of eval() as giving someone direct access to a Python command line inside your program. A malicious user can craft a string that does much more than just calculate a value.

- Technically, a hacker could run: `__import__('os').system('rm -rf /')` that would wipe out the system.

- Debuggability & Performance: Errors in strings are harder to catch (no syntax highlighting). Dynamic compilation on the fly is slower than normal code.

> Best practice: Avoid eval() whenever possible. Look for safer alternatives (e.g. literal parsing with ast.literal_eval or writing your own small parser).


## 7. Alternatives to eval()

#### 1. For Safely Evaluating Literals: ast.literal_eval
If you just need to parse a string containing a Python literal (like a list, dictionary, number, or string) from an untrusted source, the ast module is your best friend. literal_eval will only evaluate these safe literals and will raise an error if the string contains anything else, like function calls or operations.

```python
import ast

user_string_ok = "[1, 'hello', {'a': 1}]"
user_string_bad = "__import__('os').system('clear')"

# This is safe and works perfectly
my_list = ast.literal_eval(user_string_ok)
print(my_list) # Output: [1, 'hello', {'a': 1}]

# This will raise a ValueError because it's not a literal
try:
    ast.literal_eval(user_string_bad)
except ValueError as e:
    print(f"Error: {e}") # Error: malformed node or string
```

#### 2. Mapping Strings to Functions with a Dictionary
If you want to allow a user to choose an operation, don't use eval(). Use a dictionary to map their input to a safe, pre-approved function.

```python
import operator

# A safe mapping of allowed operations
allowed_operations = {
    "add": operator.add,
    "subtract": operator.sub,
    "multiply": operator.mul,
}

x = 10
y = 5
user_choice = "add" # This could come from user input

# Safely get the function from the dictionary and call it
if user_choice in allowed_operations:
    operation_func = allowed_operations[user_choice]
    result = operation_func(x, y)
    print(f"Result: {result}") # Result: 15
else:
    print("Invalid operation!")
```

---